## <span style='color:#ff5f27'> 📝 Imports

In [ ]:
# !pip install -r requirements.txt --quiet

In [ ]:
import pandas as pd
import numpy as np
import great_expectations as ge
from great_expectations.core import ExpectationSuite, ExpectationConfiguration

from features.users import generate_users
from features.videos import generate_video_content
from features.interactions import generate_interactions, generate_user_interactions_window_agg, generate_video_interactions_window_agg
from streaming import config

## <span style='color:#ff5f27'> ⚙️ Data Generation</span>


### <span style='color:#ff5f27'> 👥 Users Data Generation</span>

In [ ]:
user_data = generate_users(
    config.USERS_AMOUNT_HISTORICAL,
    historical=True,
)

data_users_df = pd.DataFrame(user_data)
data_users_df.registration_date = data_users_df.registration_date.apply(pd.to_datetime)
data_users_df.head()

In [ ]:
data_users_df.shape

### <span style='color:#ff5f27'> 🎥 Content Data Generation</span>


In [ ]:
# Generate data for videos
video_data = generate_video_content(
    config.VIDEO_AMOUNT_HISTORICAL, 
    historical=True,
)

data_video_df = pd.DataFrame(video_data)
data_video_df.upload_date = data_video_df.upload_date.apply(pd.to_datetime)

data_video_df.head()

### <span style='color:#ff5f27'> 🔗 Interactions Generation</span>


In [ ]:
# Generate interactions
interactions = generate_interactions(
    config.INTERACTIONS_AMOUNT_HISTORICAL, 
    user_data, 
    video_data,
)

data_interactions_df = pd.DataFrame(interactions)
data_interactions_df.interaction_date = data_interactions_df.interaction_date.apply(pd.to_datetime)
data_interactions_df.head()

In [ ]:
user_interactions_window_agg = generate_user_interactions_window_agg(
    config.INTERACTIONS_AMOUNT_HISTORICAL, 
    user_data, 
    video_data,
)

In [ ]:
user_interactions_window_agg_df = pd.DataFrame(user_interactions_window_agg)
user_interactions_window_agg_df.window_end_time = user_interactions_window_agg_df.window_end_time.apply(pd.to_datetime)
user_interactions_window_agg_df

In [ ]:
video_interactions_window_agg = generate_video_interactions_window_agg(    
    config.INTERACTIONS_AMOUNT_HISTORICAL, 
    user_data, 
    video_data,
)

In [ ]:
video_interactions_window_agg_df = pd.DataFrame(video_interactions_window_agg)
video_interactions_window_agg_df.window_end_time = video_interactions_window_agg_df.window_end_time.apply(pd.to_datetime)
video_interactions_window_agg_df

## <span style="color:#ff5f27">👮🏻‍♂️ Great Expectations </span>

In [ ]:
# Create a Great Expectations DataFrame from the pandas DataFrame
ge_users_df = ge.from_pandas(data_users_df)

# Initialize the expectation suite
expectation_suite_users = ge_users_df.get_expectation_suite()
expectation_suite_users.expectation_suite_name = "user_data_suite"

# Expectation: Age should be between 0 and 120
expectation_suite_users.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={"column": "age", "min_value": 12, "max_value": 100}
    )
)

# Expectations: Columns should not have null values
for column in ge_users_df.columns:
    expectation_suite_users.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_values_to_not_be_null",
            kwargs={"column": column}
        )
    )

# Expectation: Gender should only contain specific values
expectation_suite_users.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_distinct_values_to_be_in_set",
        kwargs={"column": "gender", "value_set": ["Male", "Female", "Other"]}
    )
)

In [ ]:
# Create a Great Expectations DataFrame from the pandas DataFrame
ge_video_df = ge.from_pandas(data_video_df)

# Initialize the expectation suite
expectation_suite_videos = ge_video_df.get_expectation_suite()
expectation_suite_videos.expectation_suite_name = "video_data_suite"

# Expectation: Views, Likes, and Video Length should be non-negative
for column in ["views", "likes", "video_length"]:
    expectation_suite_videos.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_values_to_be_between",
            kwargs={"column": column, "min_value": 0, "max_value": None}
        )
    )

# Expectation: Valid date format for upload_date
expectation_suite_videos.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_dateutil_parseable",
        kwargs={"column": "upload_date"}
    )
)

In [ ]:
# Create a Great Expectations DataFrame from the pandas DataFrame
ge_interactions_df = ge.from_pandas(data_interactions_df)

# Initialize the expectation suite
expectation_suite_interactions = ge_interactions_df.get_expectation_suite()
expectation_suite_interactions.expectation_suite_name = "interactions_data_suite"

# Expectations: Non-null values in all columns
for column in ge_interactions_df.columns:
    expectation_suite_interactions.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_values_to_not_be_null",
            kwargs={"column": column}
        )
    )

# Expectation: Validate interaction types
expectation_suite_interactions.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_distinct_values_to_be_in_set",
        kwargs={
            "column": "interaction_type",
            "value_set": ['like', 'dislike', 'view', 'comment', 'share', 'skip']
        }
    )
)

# Expectation: Positive watch time
expectation_suite_interactions.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={"column": "watch_time", "min_value": 0}
    )
)

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

## <span style="color:#ff5f27">🪄 Feature Group Creation </span>


In [ ]:
users_fg = fs.get_or_create_feature_group(
    name="users",
    description="Users data.",
    version=1,
    primary_key=["user_id"],
    partition_key=["registration_month"],
    event_time="registration_date",
    online_enabled=True,
    expectation_suite=expectation_suite_users,
)

users_fg.insert(data_users_df)
print('Done ✅')

In [ ]:
videos_fg = fs.get_or_create_feature_group(
    name="videos",
    description="Videos data.",
    version=1,
    primary_key=["video_id"],
    partition_key=["upload_month"],
    online_enabled=True,
    event_time="upload_date",
    expectation_suite=expectation_suite_videos,
)

videos_fg.insert(data_video_df)
print('Done ✅')

In [ ]:
interactions_fg = fs.get_or_create_feature_group(
    name="interactions",
    description="Interactions data.",
    version=1,
    primary_key=["interaction_id", "user_id", "video_id"],
    partition_key = ["interaction_day"],
    online_enabled=True,
    event_time="interaction_date",
    expectation_suite=expectation_suite_interactions,
)

interactions_fg.insert(data_interactions_df)
print('Done ✅')

In [ ]:
user_window_agg_1h_fg = fs.get_or_create_feature_group(
    "user_window_agg_1h",
    version=1,
    statistics_config=False,
    primary_key=["user_id"],
    partition_key=["interaction_day"],
    event_time="window_end_time",
    online_enabled=True,
    stream=True,
)

user_window_agg_1h_fg.insert(user_interactions_window_agg_df)

In [ ]:
video_window_agg_1h_fg = fs.get_or_create_feature_group(
    "video_window_agg_1h",
    version=1,
    statistics_config=False,
    primary_key=["video_id"],
    partition_key=["interaction_day"],
    event_time="window_end_time",
    online_enabled=True,
    stream=True,
)

video_window_agg_1h_fg.insert(video_interactions_window_agg_df)

## <span style="color:#ff5f27">🪄 Ranking Feature Group </span>


In [ ]:
video_interactions_df = pd.merge(
    data_interactions_df, 
    data_video_df, 
    on='video_id', 
    how='inner',
)

ranking_df = pd.merge(
    video_interactions_df, 
    data_users_df, 
    on='user_id', 
    how='inner',
)

ranking_df['label'] = np.where(
    ranking_df.interaction_type.isin(['view', 'like', 'share', 'comment']), 
    1, 
    0,
)

ranking_df.drop(
    ['interaction_id', 'interaction_type', 'watch_time', 'interaction_date', 'upload_date', 'registration_date'], 
    axis=1,
    inplace=True,
)

ranking_df.head()

In [ ]:
ranking_fg = fs.get_or_create_feature_group(
    name="ranking",
    description="Ranking Data.",
    version=1,
    primary_key=["user_id", "video_id"],
    partition_key = ["interaction_day"],    
    online_enabled=True, # why online?
)

ranking_fg.insert(ranking_df)
print('Done ✅')

---